# How to interpret questions in jupyter notebooks?



In [10]:
var fs = require('fs');
var path = require('path');
if (typeof importNotebook == 'undefined') {
    // Does this work everywhere?
    try {
        var m, co = '', vm = require('vm');
        var source = require('fs').readFileSync('How to find answers.ipynb').toString();
        var re = new RegExp('\\s*"cell_type": "code",\\s*"source":\\s*\\[\\s*([\\s\\S]*?\\$\\$.done[\\s\\S]*?)\\s*\\]\\s*,\s*', 'ig');
        while (m = re.exec(source)) { co += (co == '' ? '' : ',\n' ) + m[1] + ',"\\n"'; }
        co = '\n\$$ = get$$();\n\n' + JSON.parse('['+co+']').join('') + '\n\n;';
        global.getGlobal = () => global;
        global.get$$ = () => {
            var temp$$ = Object.assign({}, global.$$);
            return Object.assign(temp$$, {
                async: () => {},
                done: (done) => {}
            });
        };
        var tmpGlobal = Object.assign({}, global);
        var context = vm.createContext(tmpGlobal);
        (new vm.Script(co)).runInContext(context);
        $$.async();
        tmpGlobal.imports.then((done) => {
            Object.assign(global, tmpGlobal.commandRegistry);
            $$.done(done);
        });
    } catch (e) {
        $$.done(e);
    }
}

$$.done('interpreter is ready');


How to find all the notebooks and read markdown cells?

How to get markdown from all notebooks in {directory}?


In [11]:

var flatten = arr => arr.reduce((acc, val) => 
      acc.concat(Array.isArray(val) ? flatten(val) : val), []);

Array.prototype.flatten = function() {return flatten(this)};

function escapeRegExp(str) {
  return str.replace(/[\-\[\]\/\{\}\(\)\*\+\?\.\\\^\$\|]/g, "\\$&");
}

var walkSync = d => {
    var reg = d.split('*').slice(1).join('[^/]*');
    var dir = d.split('*')[0];
    return flatten(fs.readdirSync(dir)
    .filter(file => path.join(dir, file).match(reg))
    .map(file => fs.statSync(path.join(dir, file)).isDirectory()
        ? walkSync(path.join(dir, file))
        : path.join(dir, file).replace(/\\/g, '/')));
};
commandRegistry['walkSync'] = walkSync;
var notebooks = walkSync(path.resolve('**/*.ipynb'));
console.log(notebooks);
$$.done(walkSync);


How to find questions leading up to jupter cells?

This is a complex question to answer.  Perhaps using IBM Watson language alchemy?


In [12]:
// TODO: interpret markdown leading up to code results and find the resulting function in the list
// for now, the boring solution is to assume all markdown output is a question?
if (typeof markdownCache === 'undefined') {
    var markdownCache = {};
}
if (typeof commandRegistry === 'undefined') {
    var commandRegistry = {};
}
var interpretCells = (cells, notebook) => (resolve, reject) => {
    var previous = [];
    var sourceCount = 0;
    cells.forEach((cell) => {
        var source = cell.source.join('').trim();
        if(cell.cell_type == 'markdown') {
            previous[previous.length] = source;
        } else if (cell.cell_type == 'code') {
            if (typeof commandRegistry[notebook+'['+sourceCount+']'] !== 'undefined') {
                markdownCache[notebook+'['+sourceCount+']'] = previous;
                previous = [];
            }
            sourceCount++;
        }
    });
    resolve(cells);
};
var cells = [
    {cell_type: 'code', source: []},
    {cell_type: 'markdown', source: ["test for errors"]},
    {cell_type: 'code', source: ["(function (err) { throw error; })"]}
];
commandRegistry['How to test import[1]'] = cells[2].source.join('').trim();
interpretCells(cells, 'How to test import')(() => {}, () => {});
if(markdownCache['How to test import[1]'][0] == cells[1].source) {
    commandRegistry['interpretCells'] = interpretCells;
    $$.done(interpretCells);
}



How to interpret a jupyter {directory}?

In [13]:
var fs = require('fs');
try {
var interpretDirectory = (dirname, onFileContent, onError) => {
    var filenames = walkSync(dirname);
    console.log(filenames);
    var interpretations = filenames.map(notebook => {
        return getCells(notebook, ['markdown', 'code'])
            .then((cells, notebook) => interpretCells(cells, notebook));
    });
    $$.done(interpretations);
    return runAllPromises(Promise.all(interpretations));
}
commandRegistry['interpretDirectory'] = interpretDirectory;
interpretDirectory(path.resolve('**/How to interpret questions.ipynb'))
    .then((results) => {
        console.log(results);
        $$.done(interpretDirectory);
    });
}catch(e) {
    $$.done(e);
}
